# [bokeh User Guide -- Quickstart](https://docs.bokeh.org/en/latest/docs/user_guide/quickstart.html#userguide-quickstart)

## 準備

- 適切な Anaconda の仮想環境を起動し、実習に必要なライブラリと `bokeh` のサンプルデータをインストールする。

    - `conda install scipy pandas notebook bokeh`

    - `bokeh sampledata`

In [16]:
from bokeh.plotting import *
from bokeh.layouts import *

# グラフィックスの表示先をノートブックに指定する
output_notebook()

Loading BokehJS ...

In [8]:
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# output_notebook の前に reset_output しないと空のタブが表示される

p = figure(title='simple line chart', x_axis_label='x', y_axis_label='y')
p.line(x, y, legend_label='Temp.', line_width=2)  # legend は古い。legend_label を使うこと
show(p)

In [9]:
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

p = figure(
    tools="pan,box_zoom,reset,save",
    title="log axis example",
    y_axis_type="log", y_range=[0.001, 10**11],
    x_axis_label='sections', y_axis_label='particles')

p.line(x, x,    legend_label='y=x')
p.circle(x, x,  legend_label='y=x', fill_color='white', size=8)
p.line(x, y0,   legend_label='y=x^2', line_width=3)
p.line(x, y1,   legend_label='y=10^x', line_color='red')
p.circle(x, y1, legend_label='y=10^x', fill_color='red', line_color='red', size=6)
p.line(x, y2,   legend_label='10^x^2', line_color='orange', line_dash='4 4')
show(p)

# More examples

## Vectorized colors and sizes

Numpy 配列を用いてベクトル化された計算を利用する。

In [10]:
import numpy as np

N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    '#%02x%02x%02x' % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

p = figure(tools=TOOLS, x_range=(0, 100), y_range=(0, 100))
p.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
show(p)

## Linked panning

出力された複数の図をパン操作について連携させた UI を提供する。

**やってみよう**: いずれかの図を左右にドラッグしてみよう。

In [21]:
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = y0 + y1

s1 = figure(width=250, plot_height=250, title='y = sin(x)')
s1.circle(x, y0, size=10, color='navy', alpha=0.5)

s2 = figure(width=250, height=250, x_range=s1.x_range, title='y = cos(x)')
s2.triangle(x, y1, size=10, color='firebrick', alpha=0.5)

s3 = figure(width=250, height=250, x_range=s1.x_range, title='y = sin(x) + cos(x)')
s3.square(x, y2, size=10, color='olive', alpha=0.5)

p = gridplot([[s1, s2, s3]], toolbar_location=None)

show(p)

## Linked brushing

出力された複数の図をブラシ操作について連携させた UI を提供する。

**やってみよう**: 一方の図で矩形選択ツールか投げ縄ツールを利用してみよう。

In [23]:
from bokeh.models import ColumnDataSource

N = 300
x = np.linspace(0, 4*np.pi, N)
y0, y1 = np.sin(x), np.cos(x)

source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

left = figure(tools=TOOLS, width=350, height=350, title='y=sin(x)')
left.circle('x', 'y0', source=source)

right = figure(tools=TOOLS, width=350, height=350, title='y=cos(x)',
               x_range=left.x_range, y_range=left.y_range)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])
show(p)

# Datetime axes

In [13]:
from bokeh.sampledata.stocks import AAPL

aapl = np.array(AAPL['adj_close'])                        # アップルの株価の終り値
aapl_dates = np.array(AAPL['date'], dtype=np.datetime64)  # 日付

window_size = 30
window = np.ones(window_size) / float(window_size)  # 30日分の均等重みを利用して畳み込み計算
aapl_avg = np.convolve(aapl, window, 'same')

p = figure(plot_width=800, plot_height=350, x_axis_type='datetime')
p.circle(aapl_dates, aapl, size=4, color='darkgrey', alpha=0.2, legend_label='終り値')
p.line(aapl_dates, aapl_avg, color='navy', legend_label='一ヶ月平均')

p.title.text = 'アップルの株価の一ヶ月平均'
p.legend.location = 'top_left'
p.grid.grid_line_alpha = 0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.ygrid.band_fill_color = 'olive'
p.ygrid.band_fill_alpha = 0.1

show(p)